# Automatic Source Retrieval

Done:
- Ontario
- Manitoba
- British Columbia

TODO:
- Nova Scotia
- Northwest Territories
- Nunavut
- Yukon
- Alberta
- Saskatchewan
- Prince Edward Island
- Quebec
- New Brunswick
- Newfoundland and Labrador

In [1]:
!pip install bs4
!pip install feedparser

You are using pip version 10.0.1, however version 20.2b1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.
You are using pip version 10.0.1, however version 20.2b1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


In [2]:
import requests
import urllib.request
import datetime
from bs4 import BeautifulSoup
import pandas as pd
import feedparser
from datetime import date

In [30]:
today = date.today()
today_str = str(today).replace('-', '%2F')

print(today_str)
country = 'Canada'
src_cat = 'Government Website'
columns = ['start_date', 'country', 'region', 'subregion', 'source_url', 'source_category', 'source_title', 'source_full_text']

2020%2F06%2F04 04:08:12.378860


# Ontario

Since Ontario shows the most recent news on the first page, the range
will need to continue to be expanded to capture all posts. Generally ~4 pages capture ~2 weeks.

In [15]:
%%time

base_url = 'https://news.ontario.ca/en/search?content_type=all&utf8=%E2%9C%93&date_range_end=' + today_str + '&date_range_start=2020%2F01%2F01&date_select=desc&page='
targets = [base_url + str(i) for i in range(1,4)]

region = 'Ontario'
subregion = ''

# Specific structure for news.contario.ca/archive
links = []
page = 1
while True:
    print('Searching page ', page)
    target = base_url + str(page)
    
    response = requests.get(target)
    soup = BeautifulSoup(response.text, "html.parser")
    articles = soup.findAll('article')
    
    if len(articles) == 0:
        print('No articles found.')
        break

    for article in articles:
        smallersoup = BeautifulSoup(str(article), "html.parser")
        link = smallersoup.findAll('a')[0]['href']
        title = smallersoup.findAll('a')[0].string
        pub_date = datetime(smallersoup.time.string)

        response = requests.get(link)
        linksoup = BeautifulSoup(response.text, "html.parser")
        full_text = linksoup.article.text

        row = [pub_date, country, region, subregion, link, src_cat, title, full_text]
        links.append(row)
    
    page += 1
            

Searching page  1
Searching page  2
Searching page  3
Searching page  4
Searching page  5
Searching page  6
Searching page  7
Searching page  8
Searching page  9
Searching page  10
Searching page  11
Searching page  12
Searching page  13
Searching page  14
Searching page  15
Searching page  16
Searching page  17
Searching page  18
Searching page  19
Searching page  20
Searching page  21
Searching page  22
Searching page  23
Searching page  24
Searching page  25
Searching page  26
Searching page  27
Searching page  28
Searching page  29
Searching page  30
Searching page  31
Searching page  32
Searching page  33
Searching page  34
Searching page  35
Searching page  36
Searching page  37
Searching page  38
Searching page  39
Searching page  40
Searching page  41
Searching page  42
Searching page  43
Searching page  44
Searching page  45
Searching page  46
Searching page  47
Searching page  48
Searching page  49
Searching page  50
Searching page  51
Searching page  52
Searching page  53
Se

In [16]:
df = pd.DataFrame(links, columns=columns)

In [17]:
df.shape

(892, 8)

In [ ]:
df['start_date'] = pd.to_datetime(df['start_date']).dt.date

In [21]:
df.to_csv('sources/ontario.csv')

# Manitoba

Retrieve all news releases in 2020 for the Province of Manitoba.

In [22]:
url_base = 'https://news.gov.mb.ca'
targets = [url_base + '/news/index.html?month=' + str(i) + '&year=2020&day=01&bgnG=GO&d=' for i in range(1,12)]

In [23]:
%%time

region = 'Manitoba'
subregion = ''

links = []
for target in targets:
    print(target)
    if target.startswith(url_base): #manitoba
        response = requests.get(target)
        soup = BeautifulSoup(response.text, "html.parser")
        items = soup.findAll("div", {"class": "maincontent"})
        smallersoup = BeautifulSoup(str(items), "html.parser")
        for article in smallersoup.findAll('h2'):
            a = article.a
            relative_link = a['href']
            link = url_base + relative_link.split('..')[-1]
            title = a.string
        
            response = requests.get(link)
            linksoup = BeautifulSoup(response.text, "html.parser")
            
            date_text = linksoup.findAll("span", {"class": "article_date"})[0].string
            date = pd.to_datetime(date_text, format='%B %d, %Y')
            pub_date = date.strftime('%m/%d/%Y')
            
            full_text = linksoup.findAll("div", {"class": ""})[0].text

            
            row = [pub_date, country, region, subregion, link, src_cat, title, full_text]
            links.append(row)
            
            # Get this link and copy full text

https://news.gov.mb.ca/news/index.html?month=1&year=2020&day=01&bgnG=GO&d=
https://news.gov.mb.ca/news/index.html?month=2&year=2020&day=01&bgnG=GO&d=
https://news.gov.mb.ca/news/index.html?month=3&year=2020&day=01&bgnG=GO&d=
https://news.gov.mb.ca/news/index.html?month=4&year=2020&day=01&bgnG=GO&d=
https://news.gov.mb.ca/news/index.html?month=5&year=2020&day=01&bgnG=GO&d=
https://news.gov.mb.ca/news/index.html?month=6&year=2020&day=01&bgnG=GO&d=
https://news.gov.mb.ca/news/index.html?month=7&year=2020&day=01&bgnG=GO&d=
https://news.gov.mb.ca/news/index.html?month=8&year=2020&day=01&bgnG=GO&d=
https://news.gov.mb.ca/news/index.html?month=9&year=2020&day=01&bgnG=GO&d=
https://news.gov.mb.ca/news/index.html?month=10&year=2020&day=01&bgnG=GO&d=
https://news.gov.mb.ca/news/index.html?month=11&year=2020&day=01&bgnG=GO&d=
CPU times: user 18.3 s, sys: 550 ms, total: 18.8 s
Wall time: 8min 48s


In [25]:
df = pd.DataFrame(links, columns=columns)
df.to_csv('sources/manitoba.csv')
df.shape

(355, 7)

## British Columbia

In [27]:
#    'https://news.gov.bc.ca/Search?fromDate=2020%2F01%2F01&toDate=2020%2F05%2F26&q='
region = 'British Columbia'
subregion = ''

query_url = 'https://news.gov.bc.ca/Search?FromDate=01/01/2020&Page='

In [28]:
links = []

In [29]:
page = 1
while True:
    print("Page ", page)
    target = query_url + str(page)
    response = requests.get(target)
    soup = BeautifulSoup(response.text, "html.parser")
    items = soup.findAll("div", {"class": "article"})
    
    if not items:
        # No articles found
        break
    
    for article in items:
        smallersoup = BeautifulSoup(str(article), "html.parser")

        #for article in smallersoup.findAll('div'):

        title = smallersoup.a.string

        date_text = smallersoup.findAll("div", {"class" : "item-date"})[0].string
        date = pd.to_datetime(date_text)
        pub_date = date.strftime('%m/%d/%Y')

        link = smallersoup.a['href']

        response = requests.get(link)
        linksoup = BeautifulSoup(response.text, "html.parser")
        get_article = linksoup.findAll("article")
        if get_article:
            full_text = get_article[0].text
        else:
            print("Couldn't retrieve full text for link: ", link)
            full_text = ""

        row = [pub_date, country, region, subregion, link, src_cat, title, full_text]
        links.append(row)

    page += 1

Page  1
Page  2
Page  3
Page  4
Page  5
Page  6
Page  7
Page  8
Page  9
Page  10
Page  11
Page  12
Page  13
Page  14
Page  15
Page  16
Page  17
Page  18
Page  19
Couldn't retrieve full text for link:  https://news.gov.bc.ca/factsheets/bc-takes-steps-to-support-people-businesses-during-covid-19-pandemic
Page  20
Page  21
Page  22
Page  23
Page  24
Page  25
Page  26
Page  27
Page  28
Page  29
Page  30
Page  31
Page  32
Page  33
Page  34
Page  35
Page  36
Page  37
Page  38
Page  39
Page  40
Page  41
Page  42
Page  43
Page  44
Page  45
Page  46
Page  47
Page  48
Page  49
Page  50
Page  51
Page  52
Page  53
Page  54
Page  55
Page  56
Page  57
Page  58
Page  59
Page  60
Page  61
Page  62
Page  63
Page  64
Page  65
Page  66
Page  67
Page  68
Page  69
Page  70
Page  71
Page  72
Page  73
Page  74
Page  75


In [31]:
df = pd.DataFrame(links, columns=columns)
df.to_csv('sources/britishcolumbia.csv')
df.shape

(739, 8)